<a href="https://colab.research.google.com/github/jjyjung/ai/blob/gh-pages/tf_keras_DNN_BMI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers  #모듈(변수나 함수를 포함)만 불러오기

# BMI 데이터를 읽어 들이고 정규화하기
csv = pd.read_csv("bmi.csv")

# 몸무게와 키 데이터
csv["weight"] /= 100   #normalization
csv["height"] /= 200   #normalization
X = csv[["weight", "height"]].as_matrix()
X = csv.iloc[:,0:2]

# 레이블
bclass = {"thin":[1,0,0], "normal":[0,1,0], "fat":[0,0,1]}
y = np.empty((20000,3))     # 2000x3 크기의 다차원 벡터 생성
for i, v in enumerate(csv["label"]):
    y[i] = bclass[v]        #"thin'이면, y[i]=[1,0,0] 와 같이 할당
    
# 훈련 전용 데이터와 테스트 전용 데이터로 나누기
X_train, y_train = X[0:15000], y[0:15000]
X_test,  y_test  = X[15000:20000], y[15000:20000]

In [ ]:
# 모델 구조 정의하기
model = tf.keras.Sequential()  #순차적 계층화 준비
model.add(layers.Dense(8, input_shape=(2,)))  #입력 2개로부터 전달받는 8개 노드의 layer 생성
model.add(layers.Activation('relu'))  #ReLU 활성화함수 채택
model.add(layers.Dropout(0.1))        #dropout ratio=10% (배치 훈련시 10% arc 무시)

model.add(layers.Dense(4))         #4개 노드의 layer 생성
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.1))

model.add(layers.Dense(3))
model.add(layers.Activation('softmax')) #분류(classification)을 위해 softmax 함수 사용

# 모델 구축하기
model.compile(
    loss='categorical_crossentropy',  #다중 교차엔트로피
    optimizer="rmsprop",   #최적화 기법 중 하나
    metrics=['accuracy'])  #정확도 측정

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
# 데이터 훈련하기
hist = model.fit(
    X_train, y_train,
    batch_size=100,  #100개에 한 번씩 업데이터 실행
    epochs=50,       #훈련 데이터셋을 총 20회 반복 실험. 단, 조기중지될 수 있음
    validation_split=0.2,  
      #validation data 분할 비율. 즉, 15000개 중에서 10%인 1500개를 validation용으로 분할
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)],  
      #'val_loss'를 monitor하여 감소하면 한 번 더(1) 참고 조기중지
    verbose=1)  #전 과정을 화면에 출력(1) 또는 미출력(0) 모드

# 테스트 데이터로 평가하기
score = model.evaluate(X_test, y_test)
print('test_loss: ', score[0])
print('test_acc: ', score[1])

Train on 12000 samples, validate on 3000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
12000/12000 [==============================] - 0s 29us/sample - loss: 2.6509 - acc: 0.3069 - val_loss: 1.1011 - val_acc: 0.3237
Epoch 2/50
12000/12000 [==============================] - 0s 12us/sample - loss: 1.8506 - acc: 0.4616 - val_loss: 0.9341 - val_acc: 0.4837
Epoch 3/50
12000/12000 [==============================] - 0s 12us/sample - loss: 1.7379 - acc: 0.4695 - val_loss: 0.8949 - val_acc: 0.4897
Epoch 4/50
12000/12000 [==============================] - 0s 12us/sample - loss: 1.7607 - acc: 0.4678 - val_loss: 0.8671 - val_acc: 0.5030
Epoch 5/50
12000/12000 [==============================] - 0s 12us/sample - loss: 1.7244 - acc: 0.4929 - val_loss: 0.8300 - val_acc: 0.6293
Epoch 6/50
12000/12000 [==============================] - 0s 12us/sample - loss: 1.5035 - acc: 0.5615 - val_loss: 0.8162 - val_acc: 0.6233
Epoch 7/50
12000/12000 [==============================] - 0s 12us/sam

In [ ]:
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')

acc_ax.plot(hist.history['acc'], 'b', label='train acc')
acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()

<Figure size 640x480 with 2 Axes>

In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 24        
_________________________________________________________________
activation (Activation)      (None, 8)                 0         
_________________________________________________________________
dropout (Dropout)            (None, 8)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 36        
_________________________________________________________________
activation_1 (Activation)    (None, 4)                 0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 4)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 15        
__________

In [ ]:
model.save("dnn_BMI.h5")
print("Saved model to disk.")

#load and evaluate the saved model
from numpy import loadtxt
from tensorflow.python.keras.models import load_model

#load model
loaded_model = load_model("dnn_BMI.h5")
model.summary()

score = model.evaluate(X_test, y_test)
print('test_loss: ', score[0])
print('test_acc: ', score[1])

Saved model to disk.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 24        
_________________________________________________________________
activation (Activation)      (None, 8)                 0         
_________________________________________________________________
dropout (Dropout)            (None, 8)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 36        
_________________________________________________________________
activation_1 (Activation)    (None, 4)                 0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 4)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 

In [ ]:
model.save("dnn_bmi.h5")
print("Saved model to disk.")

#load and evaluate the saved model
from numpy import loadtxt
from tensorflow.python.keras.models import load_model

#load model
loaded_model = load_model("dnn_bmi.h5")
model.summary()

score = model.evaluate(X_test, y_test)
print('test_loss: ', score[0])
print('test_acc: ', score[1])

Saved model to disk.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 24        
_________________________________________________________________
activation (Activation)      (None, 8)                 0         
_________________________________________________________________
dropout (Dropout)            (None, 8)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 36        
_________________________________________________________________
activation_1 (Activation)    (None, 4)                 0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 4)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 

In [ ]:
#X_test의 예측 클래스 확인하기
y_pred = model.predict_classes(X_test)
y_pred[:10]

array([2, 0, 1, 0, 1, 1, 2, 0, 2, 1], dtype=int64)

In [ ]:
#X_test의 클래스별 예측 확률 확인하기
y_pred_prob = model.predict_proba(X_test)
y_pred_prob

array([[2.4848736e-05, 4.0614423e-01, 5.9383088e-01],
       [9.0394133e-01, 9.6058697e-02, 5.7913604e-28],
       [1.5519175e-04, 7.5911045e-01, 2.4073437e-01],
       ...,
       [9.8674959e-01, 1.3250425e-02, 0.0000000e+00],
       [8.9777589e-01, 1.0222407e-01, 4.8346007e-27],
       [1.3000222e-06, 4.7824937e-01, 5.2174932e-01]], dtype=float32)

In [ ]:
y_pred = model.predict_classes(X_test[0:5])
y_pred

array([2, 0, 1, 0, 1], dtype=int64)

In [ ]:
X_test[0:5]

,height,weight
15000,138,55
15001,152,36
15002,183,72
15003,198,51
15004,149,47


In [ ]:
y_pred = model.predict_classes(X_test[[1]])
y_pred

KeyError: "None of [Int64Index([1], dtype='int64')] are in the [columns]"

In [ ]:
X_test[[1]]

In [ ]:
X_test[1]

In [ ]:
X_new = [80, 175]
X_new[0]/=100
X_new[1]/=200
X_new

In [ ]:
y_pred = model.predict_classes(np.array([X_new,]))
y_pred_prob = model.predict_proba(np.array([X_new,]))
print(y_pred, y_pred_prob)

In [ ]:
def predict_bmi(X_new):
    y_pred = model.predict_classes(np.array([X_new,]))
    y_pred_prob = model.predict_proba(np.array([X_new,]))
    print(y_pred, y_pred_prob)

In [ ]:
X_mean = [X_train[:,0].mean(), X_train[:,1].mean()] #normal (1)
X_min = [X_train[:,0].min(), X_train[:,1].min()] #normal (1)
X_max = [X_train[:,0].max(), X_train[:,1].max()] #normal (1)
X_min_max = [X_train[:,0].min(), X_train[:,1].max()]  #thin (0)
X_max_min = [X_train[:,0].max(), X_train[:,1].min()]  #fat (2)
print("X_mean : ", X_mean)
print("X_min : ", X_min)
print("X_max : ", X_max)
print("X_min_max : ", X_min_max)
print("X_max_min : ", X_max_min)

In [ ]:
predict_bmi(X_mean)
predict_bmi(X_min)
predict_bmi(X_max)
predict_bmi(X_min_max)
predict_bmi(X_max_min)